# Parse SD monthly voter registration report

Uses:
- `requests` to get the PDF from [the Secretary of State's website](https://sdsos.gov/elections-voting/upcoming-elections/voter-registration-totals/voter-registration-by-county.aspx)
- `pdfplumber` to parse and validate the data

In [3]:
from io import BytesIO
from datetime import datetime
import csv

import requests
import pdfplumber

In [4]:
url = 'https://sdsos.gov/elections-voting/assets/StatewideVoterRegistrationTotals/StatewideVotersByCounty_9.3.2024.pdf'
req = requests.get(url)

In [5]:
pdf = pdfplumber.open(BytesIO(req.content))

In [6]:
# assumes a 2-page PDF with a table spanning both pages
if len(pdf.pages) != 2:
    raise Exception(f'Report page count is {len(pdf.pages)}, not 2 -- check it out: {url}')

In [7]:
lines = pdf.pages[0].extract_table() + pdf.pages[1].extract_table()

# guarding against some PDFs parsed with an extra column
lines = [x[:-1] if not x[-1] else x for x in lines]

In [8]:
lines

[['Statewide Report by County - September 3, 2024',
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None],
 ['County',
  'Democratic',
  'Libertarian',
  'Republican',
  'Independent',
  'OTH',
  'No Party\nAffiliation',
  'No Labels',
  'Inactive',
  'Total Active'],
 ['Aurora', '522', '2', '969', '223', '2', '94', '', '110', '1812'],
 ['Beadle',
  '2,506',
  '26',
  '4,969',
  '1,380',
  '41',
  '1,085',
  '1',
  '803',
  '10008'],
 ['Bennett', '684', '6', '704', '204', '2', '193', '', '198', '1793'],
 ['Bon Homme', '893', '13', '2,340', '478', '', '215', '', '298', '3939'],
 ['Brookings',
  '4,992',
  '101',
  '9,649',
  '3,579',
  '21',
  '2,662',
  '',
  '1616',
  '21004'],
 ['Brown',
  '7,277',
  '95',
  '12,649',
  '3,564',
  '11',
  '1,834',
  '1',
  '1145',
  '25431'],
 ['Brule', '903', '14', '1,714', '457', '4', '203', '', '175', '3295'],
 ['Buffalo', '525', '2', '148', '65', '3', '80', '', '130', '823'],
 ['Butte', '779', '56', '4,499', '882', '20', '602', '', '138

In [9]:
# report date is in the first line
report_date = lines.pop(0)

In [10]:
report_date = report_date[0].split('-')[-1].strip()
report_date = datetime.strptime(report_date, '%B %d, %Y').date().isoformat()

print(report_date)

2024-09-03


In [11]:
# standardizing column names
column_fixes = {
    'oth': 'other',
    'no party\naffiliation': 'npa',
    'no labels': 'no_labels',
    'npa/ind': 'npa_ind'
}

In [12]:
# headers are in the first line of the data table
headers = lines.pop(0)
headers = [column_fixes.get(x.lower(), x.lower()) for x in headers]

In [13]:
headers

['county',
 'democratic',
 'libertarian',
 'republican',
 'independent',
 'other',
 'npa',
 'no_labels',
 'inactive',
 'total active']

In [14]:
data = []

# loop over the remaining lines
for line in lines:

    first_cell = line[0].lower()

    # skip lines if repeated headers
    if 'statewide' in first_cell or 'county' in first_cell:
        continue

    # the 'Total' line in the table means we're done
    if 'total' in first_cell:
        break

    # check that the number of values matches the number of headers
    assert(len(line) == len(headers))

    # zip it up into a dict
    county_data = dict(zip(headers, line))

    # delete the aggregate value totaling everything up
    del county_data['total active']

    # cast number strings to integers
    for key in county_data:
        if key == 'county' or not county_data[key]:
            continue

        county_data[key] = int(county_data[key].replace(',', ''))

    # add the report date
    county_data['date'] = report_date
    data.append(county_data)

In [15]:
# doublecheck: 66 counties in SD
assert(len(data) == 66)

In [16]:
data[0]

{'county': 'Aurora',
 'democratic': 522,
 'libertarian': 2,
 'republican': 969,
 'independent': 223,
 'other': 2,
 'npa': 94,
 'no_labels': '',
 'inactive': 110,
 'date': '2024-09-03'}

In [17]:
# shift 'date' to the beginning of headers list for the output CSV
csv_headers = list(data[0].keys())
csv_headers = csv_headers[-1:] + csv_headers[:-1]

In [18]:
# write to file
filepath = f'data/{report_date}.csv'

with open(filepath, 'w') as outfile:
    writer = csv.DictWriter(
        outfile,
        fieldnames=csv_headers
    )

    writer.writeheader()
    writer.writerows(data)

    print(f'Wrote file: {filepath}')

Wrote file: data/2024-09-03.csv
